In [19]:
import sys
from pathlib import Path

# Ubica la raíz del proyecto; desde tests/ es el padre del cwd
PROJECT_ROOT = Path.cwd().parent

import cv2 as cv
import numpy as np

run = 6
dataset = 3

# --- Configuración de paths ---
input_dir = PROJECT_ROOT / "Database" / "data" / f"image{dataset}" / "Arandela"
output_dir = PROJECT_ROOT / "Database" / "tmp" / f"imgPreprocTry{run:02d}"
output_dir.mkdir(parents=True, exist_ok=True)

labels = ["Clavo", "Tornillo", "Tuerca", "Arandela"]

sigmas = [1.0, 2.0, 3.0, 4.0, 5.0]

for img_path in sorted(input_dir.glob("*.jpg")):
    img = cv.imread(str(img_path))
    if img is None:
        continue
    
    
    for sigmaX in sigmas:
        # --- 1) Pasar a HSV ---
        hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

        # --- 2) Rango de VERDE (OpenCV usa H en [0,179]) ---
        lower_green = np.array([35, 40, 40], dtype=np.uint8)
        upper_green = np.array([85, 255, 255], dtype=np.uint8)

        blurred = cv.GaussianBlur(hsv, (0, 0), sigmaX=sigmaX)

        out_name = f"{img_path.stem}_sigma{sigmaX:.1f}.png"
        out_path = output_dir / out_name
        cv.imwrite(str(out_path), blurred)

        # --- 3) Máscara de FONDO verde (255 = verde, 0 = no verde) ---
        mask_bg = cv.inRange(blurred, lower_green, upper_green)

        # --- 4) Máscara de OBJETO (invertimos) ---
        mask_obj = cv.bitwise_not(mask_bg)

        # (opcional) guardar para ver qué tal:
        cv.imwrite(str(output_dir / f"{img_path.stem}_{sigmaX}_mask_bg.png"), mask_bg)
        cv.imwrite(str(output_dir / f"{img_path.stem}_{sigmaX}_mask_obj.png"), mask_obj)

